In [1]:
import pandas as pd
import boto3
import awswrangler as wr
from datetime import datetime

# steps should be:
# write raw data to s3 / raw
# lambda to receive those files and validate them into s3 / validated.
# a lambda to process those validated files as they arrive and ingest into sql

# a MANUAL script to wipe the source tables and re-ingest everything in the validated s3 buckets.

# send emails out on any failed file ingestions / validations.

In [51]:
# use resource and not client, vconfusing
s3_resource = boto3.resource('s3')
extension = ("parquet", "csv")
bucket='jacobsbucket97'
boxscores_raw = list(f"{bucket}/{i.key}" for i in s3_resource.Bucket(bucket).objects.filter(Prefix = "boxscores/raw/")) # make sure it ends with /
boxscores_raw = list(filter(lambda x: x.endswith(extension), boxscores_raw))
boxscores_raw

['jacobsbucket97/boxscores/raw/boxscores-2021-10-27.parquet',
 'jacobsbucket97/boxscores/raw/boxscores-2021-11-06.parquet',
 'jacobsbucket97/boxscores/raw/boxscores-2021-11-07.parquet',
 'jacobsbucket97/boxscores/raw/boxscores-2021-11-17.parquet',
 'jacobsbucket97/boxscores/raw/boxscores-2021-11-19.parquet',
 'jacobsbucket97/boxscores/raw/boxscores-2021-11-20.parquet',
 'jacobsbucket97/boxscores/raw/boxscores-2021-11-24.parquet',
 'jacobsbucket97/boxscores/raw/boxscores-2021-12-03.parquet',
 'jacobsbucket97/boxscores/raw/boxscores-2021-12-10.parquet',
 'jacobsbucket97/boxscores/raw/boxscores-2021-12-30.parquet',
 'jacobsbucket97/boxscores/raw/boxscores-2022-01-01.parquet',
 'jacobsbucket97/boxscores/raw/boxscores-2022-01-08.parquet',
 'jacobsbucket97/boxscores/raw/boxscores-2022-01-14.parquet',
 'jacobsbucket97/boxscores/raw/boxscores-2022-01-15.parquet',
 'jacobsbucket97/boxscores/raw/boxscores-2022-01-19.csv',
 'jacobsbucket97/boxscores/raw/boxscores-2022-01-19.parquet',
 'jacobsbuck

In [50]:
# list all jimmy buckets
s3 = boto3.resource('s3')
buckets = list(s3.buckets.all())
buckets

[s3.Bucket(name='jacobs-lambda-packages'),
 s3.Bucket(name='jacobs-sqs-bucket'),
 s3.Bucket(name='jacobsbucket97'),
 s3.Bucket(name='jyablonski-lambda-bucket'),
 s3.Bucket(name='jyablonski-lambda-prac'),
 s3.Bucket(name='jyablonski-tf-cicd-bucket'),
 s3.Bucket(name='jyablonski-unhappy-bucket')]

In [71]:
s3_path = 's3://jacobsbucket97'
bucket = "jacobsbucket97"
folder = "boxscores"
initial_folder = "boxscores/raw"
s3 = boto3.client('s3')
paginator = s3.get_paginator('list_objects_v2')
pages = paginator.paginate(Bucket=bucket, Prefix=folder)

# size is in kilobytes.
# limit to 500,000 kbs.
for page in pages:
    obj_list = []
    for obj in page['Contents']:
        file_name = obj['Key'].split('/')[2]
        if 'parquet' in obj['Key']:
            print(f"Reading {s3_path}/{initial_folder}/{file_name}")
            df = wr.s3.read_parquet(f"{s3_path}/{initial_folder}/{file_name}") # read in csv, parquet, zipped shit here.
            df['file_name'] = f"{s3_path}/{folder}/validated/{file_name}"
            if len(df) > 0: # do some validation here
                wr.s3.to_parquet(
                    df = df,
                    path = f"{s3_path}/{folder}/validated/{file_name}",
                    index = False
                )
                print(f"Writing Validated File to {s3_path}/{folder}/validated/{file_name}")

Reading s3://jacobsbucket97/boxscores/raw/boxscores-2021-10-27.parquet
Writing Validated File to s3://jacobsbucket97/boxscores/validated/boxscores-2021-10-27.parquet
Reading s3://jacobsbucket97/boxscores/raw/boxscores-2021-11-06.parquet
Writing Validated File to s3://jacobsbucket97/boxscores/validated/boxscores-2021-11-06.parquet
Reading s3://jacobsbucket97/boxscores/raw/boxscores-2021-11-07.parquet
Writing Validated File to s3://jacobsbucket97/boxscores/validated/boxscores-2021-11-07.parquet
Reading s3://jacobsbucket97/boxscores/raw/boxscores-2021-11-17.parquet
Writing Validated File to s3://jacobsbucket97/boxscores/validated/boxscores-2021-11-17.parquet
Reading s3://jacobsbucket97/boxscores/raw/boxscores-2021-11-19.parquet
Writing Validated File to s3://jacobsbucket97/boxscores/validated/boxscores-2021-11-19.parquet
Reading s3://jacobsbucket97/boxscores/raw/boxscores-2021-11-20.parquet
Writing Validated File to s3://jacobsbucket97/boxscores/validated/boxscores-2021-11-20.parquet
Read

NoFilesFound: No files Found on: s3://jacobsbucket97/boxscores/raw/boxscores-2021-10-27.parquet.parquet.

In [2]:
s3_path = 's3://jacobsbucket97'
bucket = "jacobsbucket97"
folder = "boxscores"
initial_folder = "boxscores/raw"
s3 = boto3.client('s3')
paginator = s3.get_paginator('list_objects_v2')
pages = paginator.paginate(Bucket=bucket, Prefix=initial_folder)

# size is in kilobytes.
# limit to 500,000 kbs.
for page in pages:
    obj_list = []
    for obj in page['Contents']:
        print(obj)

{'Key': 'boxscores/raw/', 'LastModified': datetime.datetime(2022, 2, 10, 4, 22, 44, tzinfo=tzutc()), 'ETag': '"d41d8cd98f00b204e9800998ecf8427e"', 'Size': 0, 'StorageClass': 'STANDARD'}
{'Key': 'boxscores/raw/boxscores-2021-10-27.parquet', 'LastModified': datetime.datetime(2022, 2, 10, 4, 23, 7, tzinfo=tzutc()), 'ETag': '"bb0ff2db7f8e277bdb7d6ec9496a2657"', 'Size': 23835, 'StorageClass': 'STANDARD'}
{'Key': 'boxscores/raw/boxscores-2021-11-06.parquet', 'LastModified': datetime.datetime(2022, 2, 10, 4, 23, 7, tzinfo=tzutc()), 'ETag': '"1ad4d8485d12f48ff8f895dea2ccc974"', 'Size': 24291, 'StorageClass': 'STANDARD'}
{'Key': 'boxscores/raw/boxscores-2021-11-07.parquet', 'LastModified': datetime.datetime(2022, 2, 10, 4, 23, 7, tzinfo=tzutc()), 'ETag': '"0c5d74ae43e6198c5b2d90b555cc7825"', 'Size': 24288, 'StorageClass': 'STANDARD'}
{'Key': 'boxscores/raw/boxscores-2021-11-17.parquet', 'LastModified': datetime.datetime(2022, 2, 10, 4, 23, 7, tzinfo=tzutc()), 'ETag': '"8a7defb292080a6aae9b34772

In [7]:
bucket = 'jacobsbucket97'
s3_resource = boto3.resource('s3')
bucket = s3_resource.Bucket(name = bucket)
s3_keys = list(s3_object.key for s3_object in bucket.objects.all())

# for i in s3_keys:
#     print(i)

In [21]:
# list all buckets
s3 = boto3.resource('s3')
buckets = list(s3.buckets.all())
buckets

[s3.Bucket(name='jacobs-lambda-packages'),
 s3.Bucket(name='jacobs-sqs-bucket'),
 s3.Bucket(name='jacobsbucket97'),
 s3.Bucket(name='jyablonski-lambda-bucket'),
 s3.Bucket(name='jyablonski-lambda-prac'),
 s3.Bucket(name='jyablonski-tf-cicd-bucket'),
 s3.Bucket(name='jyablonski-unhappy-bucket')]